In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Create an imbalnced binary classification dataset
# X, y = make_classification(n_samples=10000, n_features=8, n_classes=2, 
#                            weights=[0.9, 0.1], random_state=42, flip_y=0)

X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

In [3]:
# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

<h3>Handle Imbalance</h3>

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)

X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

<h4>Track experiments</h4>

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []
for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [10]:
# dagshub setup

import dagshub
dagshub.init(repo_owner='ranjithkumarneeruganti', repo_name='mlflow_dagshub', mlflow=True)
# dagshub.init(repo_owner='learnpythonlanguage', repo_name='mlflow_dagshub_demo', mlflow=True)

Initialized MLflow to track repo "ranjithkumarneeruganti/mlflow_dagshub"

Repository ranjithkumarneeruganti/mlflow_dagshub initialized!

In [11]:
# Initialize mlflow
mlflow.set_experiment('Anamoly Detection')
# mlflow.set_tracking_uri('http://localhost:5000')

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy':report['accuracy'], 
            'recall_class_0':report['0']['recall'],
            'recall_class_1':report['1']['recall'],
            'f1_score_macro':report['macro avg']['f1-score']
        })

        if 'XGB' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')

2024/08/23 08:29:10 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/23 08:29:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/ranjithkumarneeruganti/mlflow_dagshub.mlflow/#/experiments/0/runs/8c49bbe408f94649832c22e1eee32fc5.
2024/08/23 08:29:11 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/ranjithkumarneeruganti/mlflow_dagshub.mlflow/#/experiments/0.
2024/08/23 08:29:17 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/23 08:29:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/ranjithkumarneeruganti/mlflow_dagshub.mlflow/#/experiments/0/runs/cb131d6af1a64298ae256e37dfc8e03c.
2024/08/23 08:29:18 INFO mlflow.tracking._tracking_